In [1]:
import os

In [2]:
os.getcwd()

'e:\\STUDY\\TENSORFLOW\\Projects\\1_CNN_Project\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'e:\\STUDY\\TENSORFLOW\\Projects\\1_CNN_Project'

In [5]:
import dagshub

"""
Initialize a DagsHub repository or DagsHub-related functionality.

Initialization includes:
Creates a repository on DagsHub if it doesn’t exist yet.

If dvc flag is set, adds the DagsHub repository as a dvc remote.

If mlflow flag is set, initializes MLflow environment variables to enable 
logging experiments into the DagsHub hosted MLflow. That means that if you call 
dagshub.init() in your script, then any MLflow function called later in the script
will log to the DagsHub hosted MLflow.

"""
# dagshub.init(repo_owner='Aakash00004', repo_name='Chest-Cancer-Classification-Project', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('Learning rate', 0.01)
#   mlflow.log_metric('Accuracy', 0.8)

'\nInitialize a DagsHub repository or DagsHub-related functionality.\n\nInitialization includes:\nCreates a repository on DagsHub if it doesn’t exist yet.\n\nIf dvc flag is set, adds the DagsHub repository as a dvc remote.\n\nIf mlflow flag is set, initializes MLflow environment variables to enable \nlogging experiments into the DagsHub hosted MLflow. That means that if you call \ndagshub.init() in your script, then any MLflow function called later in the script\nwill log to the DagsHub hosted MLflow.\n\n'

In [6]:
#os.getenv("MLFLOW_TRACKING_URI")

In [7]:
# os.getenv("MLFLOW_TRACKING_USERNAME")

In [8]:
# '620c670a001ca0eca0af7e36c8140bf200e2e4ed'

os.getenv("MLFLOW_TRACKING_PASSWORD")

In [9]:
import tensorflow as tf

In [10]:
# load the trained model
model=tf.keras.models.load_model("trained_model/training/trained_model.keras")

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layers (S  (None, 224, 224, 3)      0         
 equential)                                                      
                                                                 
 model (Functional)          (None, 1)                 14840139  
                                                                 
Total params: 14,840,139
Trainable params: 125,451
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
# Update the src/cnn_classifier/entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    trained_model_path:Path
    training_data:Path
    testing_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list | tuple
    params_batch_size:int

In [13]:
# Update the src/cnn_classifier/config/configuration.
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        # Creating directory
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:

        training_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, r"data/train") )
        testing_data=Path(os.path.join(self.config.data_ingestion.unzip_dir, r"data/test") )

        eval_config=EvaluationConfig(
            trained_model_path="trained_model/training/trained_model.keras",
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            mlflow_uri="https://dagshub.com/Aakash00004/Chest-Cancer-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.INPUT_SHAPE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [15]:
# Update the src/cnn_classifier/components.
import tensorflow as tf
from pathlib import Path
import mlflow
import dagshub
import mlflow.keras
from urllib.parse import urlparse
from tensorflow.keras.utils import image_dataset_from_directory as Images

In [16]:
class Evaluation:

    def __init__(self, config:EvaluationConfig):

        self.config = config

    def get_trained_model(self):

        return tf.keras.models.load_model( self.config.trained_model_path
                                         )   
                                            
    def get_test_data(self):


        self.images_test = Images(
                            directory=self.config.testing_data,
                            labels='inferred',
                            image_size = self.config.params_image_size[:-1],
                            batch_size = self.config.params_batch_size
                            )


    def save_score(self):

        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def model_evaluation(self):

        self.trained_model = self.get_trained_model()
        self.get_test_data()
        self.score = self.trained_model.evaluate(self.images_test)
        
        self.save_score()

    def log_into_mlflow(self):

        """
        This initializes the integration with DagsHub for the specified repository.

        The mlflow=True argument ensures MLflow logs (parameters, metrics, artifacts) are 
        synchronized with the DagsHub repository i.e., it sets the registry_uri as well for 
        MLflow tracking server where runs, parameters, metrics, and artifacts will be logged..

        """
        dagshub.init(repo_owner='Aakash00004', repo_name='Chest-Cancer-Classification-Project', mlflow=True)
        #print("Mlflow tracking URI",mlflow.get_tracking_uri())
       
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print("After dagshub.init of mlflow",tracking_url_type_store)
        
        with mlflow.start_run():

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                                {"loss": self.score[0], "accuracy": self.score[1]}
                            )
            # Model registry does not work with file store
            #print("Before if statment of mlflow",tracking_url_type_store)
            if tracking_url_type_store != "file":

                print("Inside if statement of mlflow")

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.trained_model, "model", registered_model_name="Custom_VGG16_Model")

            else:

                #print("Inside else statment of mlflow")
                mlflow.keras.log_model(self.trained_model, "model")
        

In [17]:
# src/cnn_classifier/pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.model_evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-09 09:50:16,409 : INFO : common : yaml file : config\config.yaml loaded successfully]
[2025-01-09 09:50:16,409 : INFO : common : yaml file : params.yaml loaded successfully]
[2025-01-09 09:50:16,409 : INFO : common : Created directory at : artifacts]
Found 174 files belonging to 2 classes.
11/11 [==============================] - 73s 6s/step - loss: 0.6733 - accuracy: 0.6897
[2025-01-09 09:51:30,950 : INFO : common : json file saved at : scores.json]
[2025-01-09 09:51:32,402 : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as Aakash00004

[2025-01-09 09:51:32,426 : INFO : helpers : Accessing as Aakash00004]
[2025-01-09 09:51:33,757 : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/repos/Aakash00004/Chest-Cancer-Classification-Project "HTTP/1.1 200 OK"]
[2025-01-09 09:51:34,844 : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Aakash00004/Chest-Cancer-Classification-Project"

[2025-01-09 09:51:34,868 : INFO : helpers : Initialized MLflow to track repo "Aakash00004/Chest-Cancer-Classification-Project"]


Repository Aakash00004/Chest-Cancer-Classification-Project initialized!

[2025-01-09 09:51:34,891 : INFO : helpers : Repository Aakash00004/Chest-Cancer-Classification-Project initialized!]
Mlflow tracking URI https://dagshub.com/Aakash00004/Chest-Cancer-Classification-Project.mlflow
Before dagshub.init of mlflow https
After dagshub.init of mlflow https


2025/01/09 09:51:37 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Before if statment of mlflow https
Inside if statement of mlflow
[2025-01-09 09:51:44,221 : WARNING : save : Found untraced functions such as _update_step_xla, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\aakas\AppData\Local\Temp\tmp140qll37\model\data\model\assets
[2025-01-09 09:51:46,796 : INFO : builder_impl : Assets written to: C:\Users\aakas\AppData\Local\Temp\tmp140qll37\model\data\model\assets]


c:\Users\aakas\anaconda3\envs\CNN_Project\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'Custom_VGG16_Model' already exists. Creating a new version of this model...
2025/01/09 09:52:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Custom_VGG16_Model, version 4
Created version '4' of model 'Custom_VGG16_Model'.


In [18]:
mlflow_uri="https://dagshub.com/Aakash00004/Chest-Cancer-Classification-Project.mlflow"
mlflow.set_registry_uri(mlflow_uri)


In [19]:
mlflow.get_registry_uri()

'https://dagshub.com/Aakash00004/Chest-Cancer-Classification-Project.mlflow'

In [20]:
mlflow.get_tracking_uri()

'https://dagshub.com/Aakash00004/Chest-Cancer-Classification-Project.mlflow'

In [21]:
urlparse(mlflow.get_tracking_uri())

ParseResult(scheme='https', netloc='dagshub.com', path='/Aakash00004/Chest-Cancer-Classification-Project.mlflow', params='', query='', fragment='')

In [22]:
urlparse(mlflow.get_tracking_uri()).scheme 

'https'

In [23]:

dags_result=dagshub.init(repo_owner='Aakash00004', 
                         repo_name='Chest-Cancer-Classification-Project', 
                         mlflow=True)



[2025-01-09 09:52:44,736 : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/repos/Aakash00004/Chest-Cancer-Classification-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Aakash00004/Chest-Cancer-Classification-Project"

[2025-01-09 09:52:44,748 : INFO : helpers : Initialized MLflow to track repo "Aakash00004/Chest-Cancer-Classification-Project"]


Repository Aakash00004/Chest-Cancer-Classification-Project initialized!

[2025-01-09 09:52:44,756 : INFO : helpers : Repository Aakash00004/Chest-Cancer-Classification-Project initialized!]


In [24]:
dags_result 

In [25]:
if urlparse(mlflow.get_tracking_uri()).scheme  != "file":

    print("hello")

hello
